In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.style as style
import plotly as py
from plotly.offline import init_notebook_mode, iplot
import chart_studio.tools as tls
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import glob
import linearmodels

base = "C:/Users/Linchen Zhang/Desktop/fall2020/linchen_robintrack/"
indir = "C:/Users/Linchen Zhang/Desktop/fall2020/linchen_robintrack/robintrack-popularity-history/tmp/popularity_export"
clean = "C:/Users/Linchen Zhang/Desktop/fall2020/linchen_robintrack/cleanPopularity/"

In [2]:
df = pd.read_csv(base + "users_prices.csv")
df.head()

,Unnamed: 0,Date,Close,Volume,Symbol,users_holding
0,0,2018-05-02,65.910004,2240500.0,A,590.0
1,1,2018-05-03,66.339996,2365900.0,A,586.0
2,2,2018-05-04,67.000000,1330800.0,A,587.0
3,3,2018-05-07,67.389999,1468700.0,A,588.0
4,4,2018-05-08,67.370003,1916100.0,A,576.0


In [3]:
stock_num = df['Symbol'].unique().size

In [4]:
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
df = df.set_index(['Symbol','Date'])

df.columns

Index(['Unnamed: 0', 'Close', 'Volume', 'users_holding'], dtype='object')

In [6]:
df.index

MultiIndex([(   'A', '2018-05-02'),
            (   'A', '2018-05-03'),
            (   'A', '2018-05-04'),
            (   'A', '2018-05-07'),
            (   'A', '2018-05-08'),
            (   'A', '2018-05-09'),
            (   'A', '2018-05-10'),
            (   'A', '2018-05-11'),
            (   'A', '2018-05-14'),
            (   'A', '2018-05-15'),
            ...
            ('ZYXI', '2020-07-20'),
            ('ZYXI', '2020-07-21'),
            ('ZYXI', '2020-07-22'),
            ('ZYXI', '2020-07-23'),
            ('ZYXI', '2020-07-24'),
            ('ZYXI', '2020-07-27'),
            ('ZYXI', '2020-07-28'),
            ('ZYXI', '2020-07-29'),
            ('ZYXI', '2020-07-30'),
            ('ZYXI', '2020-07-31')],
           names=['Symbol', 'Date'], length=3113966)

In [7]:
df['logClose'] = np.log(df['Close'])

# Add 1 to volume to avoid dividing by 0 issue.
df['logVolume'] = np.log(df['Volume']+1)

# Add 1 to users number to avoid dividing by 0 issue.
df['logUsers'] = np.log(df['users_holding']+1)

In [8]:
df['logCloseDiff2'] = df['logClose'].diff(periods = 2)
df['logVolumeDiff2'] = df['logVolume'].diff(periods = 2)
df['logUsersDiff1'] = df['logUsers'].diff(periods = 1)

In [9]:
# x = df.loc['SPY']['logClose']
# # print(x)
# y = np.repeat(x, stock_num)
# # y

# # df['log_SPY'] = y
# y

In [10]:
df


Unnamed: 0      Close     Volume  users_holding  logClose  \
Symbol Date                                                                    
A      2018-05-02           0  65.910004  2240500.0          590.0  4.188290   
       2018-05-03           1  66.339996  2365900.0          586.0  4.194793   
       2018-05-04           2  67.000000  1330800.0          587.0  4.204693   
       2018-05-07           3  67.389999  1468700.0          588.0  4.210497   
       2018-05-08           4  67.370003  1916100.0          576.0  4.210200   
...                       ...        ...        ...            ...       ...   
ZYXI   2020-07-27     3113961  19.510000  2026900.0         3323.0  2.970927   
       2020-07-28     3113962  18.510000  1604700.0         3324.0  2.918311   
       2020-07-29     3113963  18.330000  2135900.0         3336.0  2.908539   
       2020-07-30     3113964  18.870001  1210500.0         3314.0  2.937573   
       2020-07-31     3113965  19.120001   883200.0         3286.0  2.950735   

                   logVolume  logUsers  logCloseDiff2  logVolumeDiff2  \
Symbol Date                                                             
A      2018-05-02  14.622210  6.381816            NaN             NaN   
       2018-05-03  14.676669  6.375025            NaN             NaN   
       2018-05-04  14.101292  6.376727       0.016402       -0.520918   
       2018-05-07  14.199889  6.378426       0.015704       -0.476781   
       2018-05-08  14.465803  6.357842       0.005507        0.364511   
...                      ...       ...            ...             ...   
ZYXI   2020-07-27  14.522019  8.108924      -0.035250        0.094462   
       2020-07-28  14.288448  8.109225      -0.062815        0.280665   
       2020-07-29  14.574399  8.112827      -0.062388        0.052381   
       2020-07-30  14.006545  8.106213       0.019262       -0.281903   
       2020-07-31  13.691308  8.097731       0.042196       -0.883091   

                   logUsersDiff1  
Symbol Date                       
A      2018-05-02            NaN  
       2018-05-03      -0.006791  
       2018-05-04       0.001702  
       2018-05-07       0.001699  
       2018-05-08      -0.020584  
...                          ...  
ZYXI   2020-07-27      -0.000902  
       2020-07-28       0.000301  
       2020-07-29       0.003603  
       2020-07-30      -0.006615  
       2020-07-31      -0.008482  

[3113966 rows x 10 columns]

In [11]:
# df.drop(df['Date']=='2018-05-02')
# df.head()
df = df.reset_index()
df.drop(df[df['Date'] == '2018-05-02'].index, inplace = True) 
df.drop(df[df['Date'] == '2018-05-03'].index, inplace = True) 

In [12]:
df = df.set_index(['Symbol','Date'])
df

Unnamed: 0      Close     Volume  users_holding  logClose  \
Symbol Date                                                                    
A      2018-05-04           2  67.000000  1330800.0          587.0  4.204693   
       2018-05-07           3  67.389999  1468700.0          588.0  4.210497   
       2018-05-08           4  67.370003  1916100.0          576.0  4.210200   
       2018-05-09           5  68.129997  2108500.0          580.0  4.221418   
       2018-05-10           6  68.860001  1844900.0          582.0  4.232075   
...                       ...        ...        ...            ...       ...   
ZYXI   2020-07-27     3113961  19.510000  2026900.0         3323.0  2.970927   
       2020-07-28     3113962  18.510000  1604700.0         3324.0  2.918311   
       2020-07-29     3113963  18.330000  2135900.0         3336.0  2.908539   
       2020-07-30     3113964  18.870001  1210500.0         3314.0  2.937573   
       2020-07-31     3113965  19.120001   883200.0         3286.0  2.950735   

                   logVolume  logUsers  logCloseDiff2  logVolumeDiff2  \
Symbol Date                                                             
A      2018-05-04  14.101292  6.376727       0.016402       -0.520918   
       2018-05-07  14.199889  6.378426       0.015704       -0.476781   
       2018-05-08  14.465803  6.357842       0.005507        0.364511   
       2018-05-09  14.561488  6.364751       0.010921        0.361599   
       2018-05-10  14.427936  6.368187       0.021876       -0.037867   
...                      ...       ...            ...             ...   
ZYXI   2020-07-27  14.522019  8.108924      -0.035250        0.094462   
       2020-07-28  14.288448  8.109225      -0.062815        0.280665   
       2020-07-29  14.574399  8.112827      -0.062388        0.052381   
       2020-07-30  14.006545  8.106213       0.019262       -0.281903   
       2020-07-31  13.691308  8.097731       0.042196       -0.883091   

                   logUsersDiff1  
Symbol Date                       
A      2018-05-04       0.001702  
       2018-05-07       0.001699  
       2018-05-08      -0.020584  
       2018-05-09       0.006908  
       2018-05-10       0.003436  
...                          ...  
ZYXI   2020-07-27      -0.000902  
       2020-07-28       0.000301  
       2020-07-29       0.003603  
       2020-07-30      -0.006615  
       2020-07-31      -0.008482  

[3104757 rows x 10 columns]

In [13]:
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import BetweenOLS

import statsmodels.api as sm

exog_vars = ['logCloseDiff2', 'logVolumeDiff2']
exog = sm.add_constant(df[exog_vars])



C:\Users\Linchen Zhang\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



In [14]:
# Basic PooledOLS
mod = PooledOLS(df['logUsersDiff1'], exog)
pooled_res = mod.fit()

print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:          logUsersDiff1   R-squared:                        0.0024
Estimator:                  PooledOLS   R-squared (Between):              0.0030
No. Observations:             3104757   R-squared (Within):               0.0024
Date:                Wed, Sep 30 2020   R-squared (Overall):              0.0024
Time:                        14:58:15   Log-likelihood                 8.817e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3778.4
Entities:                        6780   P-value                           0.0000
Avg Obs:                       457.93   Distribution:               F(2,3104754)
Min Obs:                       1.0000                                           
Max Obs:                       553.00   F-statistic (robust):             3778.4
                            

In [15]:
# Random Effect
mod = RandomEffects(df['logUsersDiff1'], exog)
re_res = mod.fit()

print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:          logUsersDiff1   R-squared:                        0.0024
Estimator:              RandomEffects   R-squared (Between):              0.0044
No. Observations:             3104757   R-squared (Within):               0.0024
Date:                Wed, Sep 30 2020   R-squared (Overall):              0.0017
Time:                        14:58:23   Log-likelihood                 8.868e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3692.4
Entities:                        6780   P-value                           0.0000
Avg Obs:                       457.93   Distribution:               F(2,3104754)
Min Obs:                       1.0000                                           
Max Obs:                       553.00   F-statistic (robust):             3691.1
                            

In [16]:
# Between estimator
mod = BetweenOLS(df['logUsersDiff1'], exog)
be_res = mod.fit()

print(be_res)

                         BetweenOLS Estimation Summary                          
Dep. Variable:          logUsersDiff1   R-squared:                        0.0713
Estimator:                 BetweenOLS   R-squared (Between):              0.0713
No. Observations:                6780   R-squared (Within):              -4.3896
Date:                Wed, Sep 30 2020   R-squared (Overall):             -4.3699
Time:                        14:58:28   Log-likelihood                    2156.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      260.18
Entities:                        6780   P-value                           0.0000
Avg Obs:                       457.93   Distribution:                  F(2,6777)
Min Obs:                       1.0000                                           
Max Obs:                       553.00   F-statistic (robust):             260.18
                            

In [17]:
#Fixed Effect
mod = PanelOLS(df['logUsersDiff1'], exog, entity_effects = True)
fe_res = mod.fit()

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:          logUsersDiff1   R-squared:                        0.0024
Estimator:                   PanelOLS   R-squared (Between):              0.0029
No. Observations:             3104757   R-squared (Within):               0.0024
Date:                Wed, Sep 30 2020   R-squared (Overall):              0.0024
Time:                        14:58:32   Log-likelihood                 8.887e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3678.0
Entities:                        6780   P-value                           0.0000
Avg Obs:                       457.93   Distribution:               F(2,3097975)
Min Obs:                       1.0000                                           
Max Obs:                       553.00   F-statistic (robust):             3678.0
                            

In [18]:
#Time Effect
mod = PanelOLS(df['logUsersDiff1'], exog, time_effects = True)
te_res = mod.fit()

print(te_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:          logUsersDiff1   R-squared:                        0.0026
Estimator:                   PanelOLS   R-squared (Between):              0.0032
No. Observations:             3104757   R-squared (Within):               0.0024
Date:                Wed, Sep 30 2020   R-squared (Overall):              0.0024
Time:                        14:58:37   Log-likelihood                 8.916e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3968.7
Entities:                        6780   P-value                           0.0000
Avg Obs:                       457.93   Distribution:               F(2,3104202)
Min Obs:                       1.0000                                           
Max Obs:                       553.00   F-statistic (robust):             3968.7
                            

In [19]:
# Both Fixed and time effects
mod = PanelOLS(df['logUsersDiff1'], exog, entity_effects = True, time_effects = True)
fe_te_res = mod.fit()

print(fe_te_res)

C:\Users\Linchen Zhang\AppData\Local\Continuum\anaconda3\lib\site-packages\linearmodels\panel\model.py:1376: MemoryWarning:

Using low-memory algorithm to estimate two-way model. Explicitly set low_memory=True to silence this message.  Set low_memory=False to use the standard algorithm that creates dummy variables for the smaller of the number of entities or number of time periods.



                          PanelOLS Estimation Summary                           
Dep. Variable:          logUsersDiff1   R-squared:                        0.0025
Estimator:                   PanelOLS   R-squared (Between):              0.0031
No. Observations:             3104757   R-squared (Within):               0.0024
Date:                Wed, Sep 30 2020   R-squared (Overall):              0.0024
Time:                        14:58:51   Log-likelihood                 8.988e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3861.1
Entities:                        6780   P-value                           0.0000
Avg Obs:                       457.93   Distribution:               F(2,3097423)
Min Obs:                       1.0000                                           
Max Obs:                       553.00   F-statistic (robust):             3861.1
                            

In [20]:
from linearmodels.panel import compare
print(compare({'Pooled': pooled_res,'BE':be_res, 'RE':re_res, 'FE':fe_res, 'TE':te_res, 'Panel':fe_te_res}))

                                                          Model Comparison                                                         
                                   Pooled                BE                RE                FE                TE             Panel
-----------------------------------------------------------------------------------------------------------------------------------
Dep. Variable               logUsersDiff1     logUsersDiff1     logUsersDiff1     logUsersDiff1     logUsersDiff1     logUsersDiff1
Estimator                       PooledOLS        BetweenOLS     RandomEffects          PanelOLS          PanelOLS          PanelOLS
No. Observations                  3104757              6780           3104757           3104757           3104757           3104757
Cov. Est.                      Unadjusted        Unadjusted        Unadjusted        Unadjusted        Unadjusted        Unadjusted
R-squared                          0.0024            0.0713            0.002